# Evaluation

In [1]:
from bib_dedupe.dedupe_benchmark import DedupeBenchmarker
from bib_dedupe.bib_dedupe import block, match, merge, prep
from bib_dedupe.util import BibDedupeUtil
import bib_dedupe.cluster
from asreview.data import load_data, ASReviewData
from datetime import datetime
import pandas as pd

In [2]:
bd_util = BibDedupeUtil()
merge_updated_papers = True

# for benchmark_path in reversed(bd_util.get_dataset_labels()):
for benchmark_path in bd_util.get_dataset_labels():
    if benchmark_path in ["problem_cases"]: # "problem_cases" digital_work "srsr", "depression"
         continue
    print(f"Dataset: {benchmark_path}")
    
    dedupe_benchmark = DedupeBenchmarker(benchmark_path=f"../data/{benchmark_path}")
    records_df = dedupe_benchmark.get_records_for_dedupe()
    
    # Bib-dedupe
    timestamp = datetime.now()
    records_df = prep(records_df)
    actual_blocked_df = block(records_df=records_df)
    matched_df = match(actual_blocked_df)
    duplicate_id_sets = bib_dedupe.cluster.get_connected_components(matched_df)
    merged_df = merge(records_df, duplicate_id_sets=duplicate_id_sets)
    result = dedupe_benchmark.compare_dedupe_id(records_df=records_df, merged_df=merged_df, timestamp=timestamp)
    bd_util.append_to_output(result, package_name="bib-dedupe")

    # More detailed comparison for debugging
    dedupe_benchmark.export_cases(prepared_records_df=records_df, blocked_df=actual_blocked_df, matched_df=matched_df)
    
    # ASReview
    asdata = ASReviewData(records_df)
    timestamp = datetime.now()
    merged_df = asdata.drop_duplicates()
    result = dedupe_benchmark.compare_dedupe_id(records_df=records_df, merged_df=merged_df, timestamp=timestamp)
    bd_util.append_to_output(result, package_name="asreview")
    print()
    

Dataset: digital_work
Loaded 7159 records
Prep started at 2023-12-21 20:37:31
Prep completed after: 10.86 seconds
Block started at 2023-12-21 20:37:41
Blocked    24194 pairs with {'volume', 'container_title_short', 'number'}
Blocked   106595 pairs with {'volume', 'year', 'container_title_short'}
Blocked    10645 pairs with {'author_first', 'container_title_short'}
Blocked     2677 pairs with {'author_first', 'year'}
Blocked    28210 pairs with {'volume', 'year', 'number'}
Blocked      932 pairs with {'volume', 'title_short'}
Blocked     4290 pairs with {'author_first', 'title_short'}
Blocked     5620 pairs with {'container_title_short', 'title_short'}
Blocked      763 pairs with {'pages', 'title_short'}
Blocked     1059 pairs with {'year', 'title_short'}
Blocked      111 pairs with {'abstract'}
Blocked      397 pairs with {'volume', 'pages', 'container_title_short'}
Blocked      505 pairs with {'pages', 'year', 'container_title_short'}
Blocked      405 pairs with {'volume', 'pages', 'y

/home/gerit/ownCloud/data/2021-CoLRev/bib-dedupe/bib_dedupe/dedupe_benchmark.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maybe_cases_df.loc[:, "case"] = (


Export started at 2023-12-21 20:38:12
Export completed after: 14.91 seconds
Runtime: 0:00:00


/home/gerit/.local/lib/python3.8/site-packages/asreview/data/base.py:540: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = self.df[~self.duplicated(pid)]



Dataset: diabetes
Loaded 1845 records
Prep started at 2023-12-21 20:38:28
Prep completed after: 2.65 seconds
Block started at 2023-12-21 20:38:31
Blocked     3234 pairs with {'author_first', 'year'}
Blocked     2260 pairs with {'pages', 'title_short'}
Blocked     2542 pairs with {'author_first', 'title_short'}
Blocked     2622 pairs with {'year', 'title_short'}
Blocked     3343 pairs with {'author_first', 'container_title_short'}Blocked     1863 pairs with {'volume', 'container_title_short', 'number'}

Blocked     2434 pairs with {'container_title_short', 'title_short'}
Blocked     2309 pairs with {'volume', 'title_short'}
Blocked     5422 pairs with {'volume', 'year', 'container_title_short'}
Blocked     2335 pairs with {'pages', 'year', 'container_title_short'}
Blocked     2065 pairs with {'doi'}
Blocked     1974 pairs with {'volume', 'year', 'number'}
Blocked     2126 pairs with {'volume', 'pages', 'container_title_short'}
Blocked     1339 pairs with {'pages', 'year', 'number'}
Blo

...

Export started at 2023-12-21 21:44:08
Export completed after: 25.75 seconds


/home/gerit/.local/lib/python3.8/site-packages/asreview/data/base.py:540: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = self.df[~self.duplicated(pid)]


Runtime: 0:00:00

Dataset: cytology_screening
Loaded 1856 records
Prep started at 2023-12-21 21:44:38
Prep completed after: 2.78 seconds
Block started at 2023-12-21 21:44:41
Blocked     3662 pairs with {'volume', 'year', 'container_title_short'}
Blocked     1274 pairs with {'author_first', 'year'}
Blocked      878 pairs with {'author_first', 'title_short'}
Blocked     1139 pairs with {'author_first', 'container_title_short'}
Blocked      811 pairs with {'pages', 'title_short'}
Blocked      821 pairs with {'container_title_short', 'title_short'}
Blocked      904 pairs with {'year', 'title_short'}
Blocked      887 pairs with {'volume', 'title_short'}
Blocked      773 pairs with {'volume', 'pages', 'container_title_short'}
Blocked      836 pairs with {'volume', 'pages', 'year'}
Blocked      772 pairs with {'pages', 'year', 'container_title_short'}
Blocked 4375 pairs
Blocked pairs reduced to 1106 pairs
Block completed after: 2.88 seconds
Sim started at 2023-12-21 21:44:44
Sim completed aft

/home/gerit/ownCloud/data/2021-CoLRev/bib-dedupe/bib_dedupe/dedupe_benchmark.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maybe_cases_df.loc[:, "case"] = (


Export started at 2023-12-21 21:44:47
Export completed after: 1.06 seconds
Runtime: 0:00:00

